#### This notebook is to understand the Apriori Algorithm

In [1]:
from collections import Counter,OrderedDict

In [2]:
# This Function is used to generate all the subset of any set
def genSubsets(l):
    powerSetSize = 2 ** len(l)
    powerSet = []
    for i in range(1,powerSetSize):
        tempEle = []
        for j in range(len(l)):
            binFlagInd = i & (1 << j)
            if binFlagInd:
                tempEle.append(l[j])
        powerSet.append(tempEle)
    return powerSet

In [3]:
print(genSubsets([1, 2, 3, 4,5]))

[[1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3], [4], [1, 4], [2, 4], [1, 2, 4], [3, 4], [1, 3, 4], [2, 3, 4], [1, 2, 3, 4], [5], [1, 5], [2, 5], [1, 2, 5], [3, 5], [1, 3, 5], [2, 3, 5], [1, 2, 3, 5], [4, 5], [1, 4, 5], [2, 4, 5], [1, 2, 4, 5], [3, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [1, 2, 3, 4, 5]]


In [4]:


def initPass(txList): # list of transactions, most possibly a dict
    allTx = [item for tx in txList for item in tx]
    allTx.sort()
    cntr =  OrderedDict()
    for tx in allTx:
        cntr[tx] = cntr.get(tx,0) + 1

    return cntr



In [5]:
T = [
    ['1', '3', '4'],
    ['2', '3', '5'],
    ['1', '2', '3', '5'],
    ['1','2','5']
    ]
print(initPass(T))

OrderedDict([('1', 3), ('2', 3), ('3', 3), ('4', 1), ('5', 3)])


In [6]:
# Apriori Algorithm Implementation
def genCandidate(Fk1): #Fk1 indicates F(k-1), it is a list of lists
    Ck = []
    k1  = len(Fk1[0])

    # COMBINE STEP
    for i in range(len(Fk1)-1):
        for j in range(i+1,len(Fk1)):
            f1,f2 = Fk1[i],Fk1[j]

            if f1[:len(f1)-1] == f2[:len(f2)-1] and f1[-1] < f2[-1]:
                tempC = f1 + [f2[-1]]

                # PRUNING STEP
                subset = genSubsets(tempC)
                appendSts = True
                for item in subset:
                    if (len(item) == k1 and item not in Fk1):
                        appendSts = False
                if appendSts:
                    Ck.append(tempC)
    return Ck

In [7]:
print(genCandidate([['1'], ['2'], ['3'], ['5']]))

[['1', '2'], ['1', '3'], ['1', '5'], ['2', '3'], ['2', '5'], ['3', '5']]


In [8]:
def searchInT(t,candid):
    found = True
    for eachCandid in candid:
        if eachCandid not in t:
            found = False

    return found


In [9]:
def apriori(T,minSup):
    finalSet = []
    c1 = initPass(T)
    f = [[item] for item in c1.keys() if c1[item]/len(T) >= minSup] # f1
    #print(f)
    for item in f:
        finalSet.append(item)

    while len(f) != 0:
        Ck = genCandidate(f)
        freqDict = {}
        for t in T:
            for candidate in Ck:
                if searchInT(t,candidate):
                    freqDict[tuple(candidate)] = freqDict.get(tuple(candidate),0) + 1
        # print("freqDict")
        # print(freqDict)
        f = []
        for c in freqDict.keys():
            if freqDict[c]/len(T)>= minSup:
                f.append(list(c))

        # print("f")
        # print(f)
        if len(f) != 0:
            f = sorted(f,key=lambda x : (len(x),*x))
            for item in f:
                finalSet.append(item)
    # print(finalSet)
    return finalSet

In [10]:
T = [
    ['1', '3', '4'],
    ['2', '3', '5'],
    ['1', '2', '3', '5'],
    ['1','2','5']
    ]

print(apriori(T,0.5))

[['1'], ['2'], ['3'], ['5'], ['1', '2'], ['1', '3'], ['1', '5'], ['2', '3'], ['2', '5'], ['3', '5'], ['1', '2', '5'], ['2', '3', '5']]


In [11]:
T2=[['Bread', 'Milk'],
            ['Bread', 'Diapers', 'Beer', 'Eggs'],
            ['Milk', 'Diapers', 'Beer', 'Coke'],
            ['Bread', 'Milk', 'Diapers', 'Beer'],
            ['Bread', 'Milk', 'Diapers', 'Coke']]
F=apriori(T2,0.5)

In [12]:
print(F)

[['Beer'], ['Bread'], ['Diapers'], ['Milk'], ['Beer', 'Diapers'], ['Bread', 'Diapers'], ['Bread', 'Milk'], ['Diapers', 'Milk']]


In [13]:
T3 = [
    ['beer', 'wine', 'cheese'],
    ['beer', 'potato chips'],
    ['eggs', 'flower', 'butter', 'cheese'],
    ['eggs', 'flower', 'butter', 'beer', 'potato chips'],
    ['wine', 'cheese'],
    ['potato chips'],
    ['eggs', 'flower', 'butter', 'wine', 'cheese'],
    ['eggs', 'flower', 'butter', 'beer', 'potato chips'],
    ['wine', 'beer'],
    ['beer', 'potato chips'],
    ['butter', 'eggs'],
    ['beer', 'potato chips'],
    ['flower', 'eggs'],
    ['beer', 'potato chips'],
    ['eggs', 'flower', 'butter', 'wine', 'cheese'],
    ['beer', 'wine', 'potato chips', 'cheese'],
    ['wine', 'cheese'],
    ['beer', 'potato chips'],
    ['wine', 'cheese'],
    ['beer', 'potato chips']
]

In [14]:
F=apriori(T3,0.5)
print(F)

[['beer'], ['potato chips']]


In [15]:
for i in range(0,10,1):
    F=apriori(T3,i/10)
    print(i/10)
    print(F)
    print()

0.0
[['beer'], ['butter'], ['cheese'], ['eggs'], ['flower'], ['potato chips'], ['wine'], ['beer', 'butter'], ['beer', 'cheese'], ['beer', 'eggs'], ['beer', 'flower'], ['beer', 'potato chips'], ['beer', 'wine'], ['butter', 'cheese'], ['butter', 'eggs'], ['butter', 'flower'], ['butter', 'potato chips'], ['butter', 'wine'], ['cheese', 'eggs'], ['cheese', 'flower'], ['cheese', 'potato chips'], ['cheese', 'wine'], ['eggs', 'flower'], ['eggs', 'potato chips'], ['eggs', 'wine'], ['flower', 'potato chips'], ['flower', 'wine'], ['potato chips', 'wine'], ['beer', 'butter', 'eggs'], ['beer', 'butter', 'flower'], ['beer', 'butter', 'potato chips'], ['beer', 'cheese', 'potato chips'], ['beer', 'cheese', 'wine'], ['beer', 'eggs', 'flower'], ['beer', 'eggs', 'potato chips'], ['beer', 'flower', 'potato chips'], ['beer', 'potato chips', 'wine'], ['butter', 'cheese', 'eggs'], ['butter', 'cheese', 'flower'], ['butter', 'cheese', 'wine'], ['butter', 'eggs', 'flower'], ['butter', 'eggs', 'potato chips'], [

In [32]:
def getMaxRules(inp : list[list[int]])->None:
    max_rules = len(apriori(inp,0))
    i=0
    resolution = 0.0001
    while(i<=1):
        rule_length = len(apriori(inp,i))
        if(max_rules > rule_length):
            i-=resolution
            break
        i+=resolution
    print(f"Maximum Number of rules can be found upto support value {i}: {max_rules}")

getMaxRules(T3)

Maximum Number of rules can be found upto support value 0.0499000000000004: 64


In [33]:
def getMinRules(inp : list[list[int]])->None:
    min_rules = len(apriori(inp,1))
    i=1
    resolution = 0.0001
    while(i>=0):
        rule_length = len(apriori(inp,i))
        if(min_rules < rule_length):
            i+=resolution
            break
        i-=resolution
    print(f"Minimum Number of rules can be found upto support value {i}: {min_rules}")

getMinRules(T3)

Minimum Number of rules can be found upto support value 0.5500000000000496: 0


In [18]:
#!pip install pyECLAT

In [19]:
import pandas as pd

# you simply convert the transaction list into a dataframe
data = pd.DataFrame(T3)
data

,0,1,2,3,4
0,beer,wine,cheese,None,None
1,beer,potato chips,None,None,None
2,eggs,flower,butter,cheese,None
3,eggs,flower,butter,beer,potato chips
4,wine,cheese,None,None,None
5,potato chips,None,None,None,None
6,eggs,flower,butter,wine,cheese
7,eggs,flower,butter,beer,potato chips
8,wine,beer,None,None,None
9,beer,potato chips,None,None,None


In [20]:
# we are looking for itemSETS
# we do not want to have any individual products returned
min_n_products = 2

# we want to set min support to 7
# but we have to express it as a percentage
min_support = 7/len(T3)

# we have no limit on the size of association rules
# so we set it to the longest transaction
max_length = max([len(x) for x in T3])

In [21]:
from pyECLAT import ECLAT

# create an instance of eclat
my_eclat = ECLAT(data=data, verbose=True)

# fit the algorithm
rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
                                           min_combination=min_n_products,
                                           max_combination=max_length)

100%|██████████| 8/8 [00:00<00:00, 7923.12it/s]


100%|██████████| 8/8 [00:00<00:00, 666.50it/s]


Combination 2 by 2


10it [00:00, 142.82it/s]


Combination 3 by 3


10it [00:00, 204.26it/s]


Combination 4 by 4


5it [00:00, 119.03it/s]


Combination 5 by 5


1it [00:00, 90.89it/s]


In [22]:
print(rule_supports)

{'potato chips & beer': 0.45, 'wine & cheese': 0.35}
